출처 : https://bab2min.tistory.com/570  
출처 : https://wiserloner.tistory.com/932

In [6]:
!pip install networkx
!pip install konlpy

     |████████████████████████████████| 19.4MB 12.7MB/s 
     |████████████████████████████████| 460kB 50.2MB/s 
     |████████████████████████████████| 92kB 12.6MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [7]:
import networkx
import re

### **RawSentenceReader**
file path를 읽어, Iterate할 때 문장 단위로 반환해주는 클래스


In [22]:
line = "청주 출신으로 청주고와 경찰대를 졸업한 송해영 계장은 1995년 경위로 경찰에 입문해 충북청 기획예산계장, 경무계장, 홍보계장 등을 역임했다.괴산 출신인 김경태 계장은 청주 운호고와 청주대학교를 졸업한 뒤 1993년 경찰에 입문했다."
rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
ch = rgxSplitter.split(line)
map(lambda a, b: a + b, ch[::2], ch[1::2])

청주 출신으로 청주고와 경찰대를 졸업한 송해영 계장은 1995년 경위로 경찰에 입문해 충북청 기획예산계장, 경무계장, 홍보계장 등을 역임했다.
괴산 출신인 김경태 계장은 청주 운호고와 청주대학교를 졸업한 뒤 1993년 경찰에 입문했다.


In [8]:
class RawSentenceReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s

### **TextRank**

문서 내 문장들의 TR을 구하는 클래스

**```loadSents```** 

문장을 읽어 품사 태깅 값 저장한 후, 문장 간 유사도를 비교하여 일정 이상 유사할 경우 ```dictBiCount[i,j]```에 저장하는 함수이다.    

- ```dictCount``` : 문장 저장  
- ```sentSet``` : 문장의 품사 태깅한 값 저장 (명사, 동사, 형용사만 추출)  
- ```dictBiCount``` : 현재 문장 i와 이전 문장 j의 유사도를 비교하여, threshold를 넘을 경우 ```dictBiCount[i,j]```에 유사도 값 저장    
  

유사도는 각 문장 sentSet 간의 교집합 원소 개수를 n이라 했을 때, 아래 식을 계산한 값이다.  
아래 유사도는 자카드 유사도 계산식에 따라 계산되었으며, 0에서 1 사이 값을 가진다.  
```
similarity = n / float(len(A) + len(B) - n) / (math.log(len(A)+1) * math.log(len(B)+1))
```
  
**```build```**
 
문장의 개수만큼 노드를 생성하고, i 노드와 j 노드 사이 간선에 아래 식을 계산한 값을 부여한다.  

```
weight = dictBiCount[i,j]*self.coef + (1-self.coef) # self.coef = 1.0 (미지정)
```

   
**```rank```**  
생성한 그래프를 바탕으로 각 노드의 ```pagerank```를 구한다.  


**```summarize```**  
입력 값 ratio 값에 따라 ```int(문장 개수 * ratio)``` 만큼 중요 문장을 이어붙여 반환한다. 


### **품사 태깅 (POS Tagging, part-of-speech tagging)**

각 단어의 품사를 태깅하는 프로세스.

Komoran는 형태소 분석을 지원하며, 이를 사용하여 품사 태깅 진행  
```NNG``` : 일반 명사   
```NNP``` : 고유 명사  
```VV``` : 동사  
```VA``` : 형용사  

In [27]:
sent = "청주 출신으로 청주고와 경찰대를 졸업한 송해영 계장은 1995년 경위로 경찰에 입문해 충북청 기획예산계장, 경무계장, 홍보계장 등을 역임했다."
tokenizer = lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent))
fil = tokenizer(sent)
s= list(fil)
s

[('청주', 'NNP'),
 ('출신', 'NNG'),
 ('청주', 'NNP'),
 ('고와', 'NNP'),
 ('경찰대', 'NNG'),
 ('졸업', 'NNG'),
 ('송해', 'NNP'),
 ('영', 'NNP'),
 ('계장', 'NNG'),
 ('경위', 'NNG'),
 ('경찰', 'NNG'),
 ('입문', 'NNG'),
 ('충북', 'NNP'),
 ('청', 'NNP'),
 ('기획', 'NNG'),
 ('예산', 'NNP'),
 ('계장', 'NNG'),
 ('경무', 'NNP'),
 ('계장', 'NNG'),
 ('홍보', 'NNG'),
 ('계장', 'NNG'),
 ('역임', 'NNG')]

In [116]:
class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}

        self.tfidf_vectorizer = TfidfVectorizer()
        self.tfidf_matrix = {}

    def loadSents(self, sentenceIter, tokenizer = None, similarity = 'jaccard'):
        import math
        def jaccard_similarity(a, b):
            n = len(a.intersection(b))
            return n / float(len(a) + len(b) - n) / (math.log(len(a)+1) * math.log(len(b)+1))

        def tfidf_cosine_similarity(i, j):
            from sklearn.metrics.pairwise import cosine_similarity
            return cosine_similarity(self.tfidf_matrix[i-1:i], self.tfidf_matrix[j-1:j])[0,0]
 
        if not tokenizer: rgxSplitter = re.compile('[\\s.,:;-?!()"\']+')
        sentSet = []
        for sent in filter(None, sentenceIter):
            if type(sent) == str:
                if tokenizer: s = set(filter(None, tokenizer(sent)))
                else: s = set(filter(None, rgxSplitter.split(sent)))
            else: s = set(sent)
            if len(s) < 2: continue
            self.dictCount[len(self.dictCount)] = sent
            sentSet.append(s)
        
        sents = set(filter(None, sentenceIter))
        self.tfidf_matrix = self.tfidf_vectorizer.fit_transform(sents)

        if similarity is not 'jaccard' :
            for i in range(1, len(self.dictCount)):
                for j in range(i+1, len(self.dictCount)):
                    print(i,j)
                    s = tfidf_cosine_similarity(i, j)

                    if s < self.threshold: continue
                    self.dictBiCount[i, j] = s
        else: 
            for i in range(len(self.dictCount)):
                for j in range(i+1, len(self.dictCount)):
                    s = jaccard_similarity(sentSet[i], sentSet[j])

                    if s < self.threshold: continue
                    self.dictBiCount[i, j] = s
 
    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
 
    def summarize(self, ratio = 0.333):
        r = self.rank()
        ks = sorted(r, key=r.get, reverse=True)[:int(len(r)*ratio)]
        return ' '.join(map(lambda k:self.dictCount[k], sorted(ks)))

기사 파일 경로 지정

In [10]:
import os
from konlpy.tag import Komoran
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [11]:
BASE_DIR = "/content/gdrive/My Drive/Colab Notebooks/Text-preprocessing-Data/articles"
TARGET_PATH = os.path.join(BASE_DIR,"Preprocessed-Data")
ARTICLE_MEDIA_PATH = os.path.join(BASE_DIR,"Origin-Data")
SWORDS_FILE_PATH = os.path.join(BASE_DIR, "StopWordList.txt")

In [59]:
media_list = os.listdir(ARTICLE_MEDIA_PATH)
article_list = os.listdir(os.path.join(ARTICLE_MEDIA_PATH, media_list[0]))
article_path = os.path.join(os.path.join(ARTICLE_MEDIA_PATH, media_list[0]), article_list[0])

불용어로 '있', '하', '되'를 지정하였으며 이는 품사 태깅에서 제외될 항목이다.  
rank 함수를 통해 얻은 각 문서의 TR값에 따라 정렬할 경우, 중요한 순서대로 나열된다.  



In [118]:
tr = TextRank()
print('Load...')

tagger = Komoran()
stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])


Load...


자카드 유사도 계산식을 사용하여 문장 간 유사도를 구한 경우

In [119]:

tr.loadSents(RawSentenceReader(article_path), lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent)))
print('Build...')
tr.build()
ranks = tr.rank()
for k in sorted(ranks, key=ranks.get, reverse=True)[:100]:
    print("\t".join([str(k), str(ranks[k]), str(tr.dictCount[k])]))
print(tr.summarize(0.2))

Build...
4	0.29990783994376224	 청주 출신으로 청주고와 경찰대를 졸업한 송해영 계장은 1995년 경위로 경찰에 입문해 충북청 기획예산계장, 경무계장, 홍보계장 등을 역임했다.
2	0.2507702844427132	 괴산 출신인 김경태 계장은 청주 운호고와 청주대학교를 졸업한 뒤 1993년 경찰에 입문했다.
0	0.1964789302540586	(청주=뉴스1) 조준영 기자 = 충북 경찰에서 4년 연속 2명의 총경 승진자가 나왔다.
1	0.1552487948361047	 경찰청은 7일 단행한 인사에서 김경태 충북경찰청 여성보호계장(50·일반 공채)과 송해영 충북청 홍보계장(49·경찰대 11기)을 포함한 경정 107명을 총경으로 승진 내정했다.
3	0.09759415052336164	 이후 청주 흥덕경찰서 생활안전과장과 경비교통과장, 충청 보안수사대장, 생활질서계장 등을 지냈다.
 청주 출신으로 청주고와 경찰대를 졸업한 송해영 계장은 1995년 경위로 경찰에 입문해 충북청 기획예산계장, 경무계장, 홍보계장 등을 역임했다.


TFIDF를 이용하여 코사인 유사도 계산식을 사용하여 문장 간 유사도를 구한 경우

In [117]:
tr = TextRank()
print('Load...')

tagger = Komoran()
stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])

tr.loadSents(RawSentenceReader(article_path), 
             lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent)), similarity='tfidf')
print('Build...')
tr.build()
ranks = tr.rank()
for k in sorted(ranks, key=ranks.get, reverse=True)[:100]:
    print("\t".join([str(k), str(ranks[k]), str(tr.dictCount[k])]))
print(tr.summarize(0.2))

Load...
1 2
1 3
1 4
2 3
2 4
3 4
Build...
1	0.44225940081600207	 경찰청은 7일 단행한 인사에서 김경태 충북경찰청 여성보호계장(50·일반 공채)과 송해영 충북청 홍보계장(49·경찰대 11기)을 포함한 경정 107명을 총경으로 승진 내정했다.
2	0.25517093968005683	 괴산 출신인 김경태 계장은 청주 운호고와 청주대학교를 졸업한 뒤 1993년 경찰에 입문했다.
4	0.21166056859485038	 청주 출신으로 청주고와 경찰대를 졸업한 송해영 계장은 1995년 경위로 경찰에 입문해 충북청 기획예산계장, 경무계장, 홍보계장 등을 역임했다.
0	0.04545454545454547	(청주=뉴스1) 조준영 기자 = 충북 경찰에서 4년 연속 2명의 총경 승진자가 나왔다.
3	0.04545454545454547	 이후 청주 흥덕경찰서 생활안전과장과 경비교통과장, 충청 보안수사대장, 생활질서계장 등을 지냈다.
 경찰청은 7일 단행한 인사에서 김경태 충북경찰청 여성보호계장(50·일반 공채)과 송해영 충북청 홍보계장(49·경찰대 11기)을 포함한 경정 107명을 총경으로 승진 내정했다.


### **TF-IDF를 이용한 문장 유사도 계산** 

특정 문서 내에서 단어 빈도가 높을 수록, 그리고 전체 문서들 중 그 단어를 포함한 문서가 적을 수록 TF-IDF값이 높아진다.

- TF : 특정 단어 t가 문서 d 내에서 얼마나 자주 등장하는 지 나타내는 지표
- IDF : 특정 단어가 등장한 문서의 수를 나타내는 지표인 DF에 반비례하는 수 

*|D|는 전체 문서의 수를 의미한다.*

```
tfidf(t, d, D) = tf(t, d) * idf(t, D)
```


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [44]:
import math

threshold = 0.005
dictCount = {}
dictBiCount = {}

sentenceIter = RawSentenceReader(article_path)
tokenizer = lambda sent: filter(lambda x:x not in stopword and x[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent))

if not tokenizer: rgxSplitter = re.compile('[\\s.,:;-?!()"\']+')
sentSet = []


In [48]:
sents = set(filter(None, sentenceIter))
sents

{' 경찰청은 7일 단행한 인사에서 김경태 충북경찰청 여성보호계장(50·일반 공채)과 송해영 충북청 홍보계장(49·경찰대 11기)을 포함한 경정 107명을 총경으로 승진 내정했다.',
 ' 괴산 출신인 김경태 계장은 청주 운호고와 청주대학교를 졸업한 뒤 1993년 경찰에 입문했다.',
 ' 이후 청주 흥덕경찰서 생활안전과장과 경비교통과장, 충청 보안수사대장, 생활질서계장 등을 지냈다.',
 ' 청주 출신으로 청주고와 경찰대를 졸업한 송해영 계장은 1995년 경위로 경찰에 입문해 충북청 기획예산계장, 경무계장, 홍보계장 등을 역임했다.',
 '(청주=뉴스1) 조준영 기자 = 충북 경찰에서 4년 연속 2명의 총경 승진자가 나왔다.',
 'reason@news1.'}

In [49]:

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sents)
idf = tfidf_vectorizer.idf_
print(dict(zip(tfidf_vectorizer.get_feature_names(), idf)))

{'107명을': 2.252762968495368, '11기': 2.252762968495368, '1993년': 2.252762968495368, '1995년': 2.252762968495368, '2명의': 2.252762968495368, '49': 2.252762968495368, '4년': 2.252762968495368, '50': 2.252762968495368, '7일': 2.252762968495368, 'news1': 2.252762968495368, 'reason': 2.252762968495368, '경무계장': 2.252762968495368, '경비교통과장': 2.252762968495368, '경위로': 2.252762968495368, '경정': 2.252762968495368, '경찰대': 2.252762968495368, '경찰대를': 2.252762968495368, '경찰에': 1.8472978603872037, '경찰에서': 2.252762968495368, '경찰청은': 2.252762968495368, '계장은': 1.8472978603872037, '공채': 2.252762968495368, '괴산': 2.252762968495368, '기자': 2.252762968495368, '기획예산계장': 2.252762968495368, '김경태': 1.8472978603872037, '나왔다': 2.252762968495368, '내정했다': 2.252762968495368, '뉴스1': 2.252762968495368, '단행한': 2.252762968495368, '등을': 1.8472978603872037, '보안수사대장': 2.252762968495368, '생활안전과장과': 2.252762968495368, '생활질서계장': 2.252762968495368, '송해영': 1.8472978603872037, '승진': 2.252762968495368, '승진자가': 2.252762968495368, '여성보호계장':

In [55]:
from sklearn.metrics.pairwise import cosine_similarity

val = cosine_similarity(tfidf_matrix[1:2], tfidf_matrix[2:3])[0,0]
print(val)

0.0


In [ ]:

for sent in filter(None, sentenceIter):
    if type(sent) == str:
        if tokenizer: s = set(filter(None, tokenizer(sent)))
        else: s = set(filter(None, rgxSplitter.split(sent)))
    else: s = set(sent)
    if len(s) < 2: continue
    dictCount[len(dictCount)] = sent
    sentSet.append(s)


for i in range(len(dictCount)):
    for j in range(i+1, len(dictCount)):
        s = similarity(sentSet[i], sentSet[j])
        if s < threshold: continue
        dictBiCount[i, j] = s
 